In [1]:
%run Requirements.ipynb

24/05/09 17:15:44 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.78 instead (on interface en0)
24/05/09 17:15:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/09 17:15:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='formulaOne'
)
cursor = conn.cursor()

In [7]:
df = spark.sql(
    """WITH CTE AS (SELECT 
        R.Year,
        D.DriverID,
        SUM(RS.Points) AS TotalPoints,
        MAX(SUM(RS.Points)) OVER (PARTITION BY R.Year) AS MaxPointsInYear
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/races` AS R 
    LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/results` AS RS ON RS.raceId = R.raceid 
    INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/driver_standings` AS D 
    ON R.raceId = D.raceId AND RS.driverid = D.driverid 
    GROUP BY R.Year, D.DriverID
)
SELECT 
    DriverID,
    SUM(CASE WHEN TotalPoints = MaxPointsInYear THEN 1 ELSE 0 END) AS TotalWorldChampionships
FROM CTE
GROUP BY DriverID
"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/WorldChampionsDriver')

In [8]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/WorldChampionsDriver")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS TotalRaceResults;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS TotalRaceResults ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO TotalRaceResults VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [11]:
df = spark.sql(
    """WITH CTE AS (
    SELECT 
        R.Year,
        C.ConstructorID,
        SUM(RS.Points) AS TotalPoints,
        MAX(SUM(RS.Points)) OVER (PARTITION BY R.Year) AS MaxPointsInYear
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/races` AS R 
    LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/results` AS RS ON RS.raceId = R.raceid 
    INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructor_standings` AS C ON R.raceId = C.raceId AND RS.constructorid = C.constructorid 
    GROUP BY R.Year, C.ConstructorID
)
SELECT 
    CTE.ConstructorID, C.name,
    SUM(CASE WHEN TotalPoints = MaxPointsInYear THEN 1 ELSE 0 END) AS TotalConstructorChampionships
FROM CTE AS CTE INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructors` AS C ON CTE.constructorId = C.constructorId
GROUP BY CTE.ConstructorID, C.name""")

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/WorldChampionsConstructors')


24/05/09 16:01:59 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 608504 ms exceeds timeout 120000 ms
24/05/09 16:01:59 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/09 16:02:02 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/WorldChampionsConstructors")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS TotalRaceResults;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS TotalRaceResults ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO TotalRaceResults VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [5]:
df = spark.sql(
    """WITH CTE AS (
    SELECT 
        R.Year,
        D.DriverID,
        RS.constructorId AS ConstructorID,
        SUM(RS.Points) AS TotalPoints,
        MAX(SUM(RS.Points)) OVER (PARTITION BY R.Year) AS MaxPointsInYear
    FROM 
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/races` AS R 
    LEFT JOIN 
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/results` AS RS ON RS.raceId = R.raceid 
    INNER JOIN 
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/driver_standings` AS D ON R.raceId = D.raceId AND RS.driverid = D.driverid 
    LEFT JOIN 
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructor_standings` AS C ON R.raceId = C.raceId AND RS.constructorId = C.constructorId
    GROUP BY 
        R.Year, D.DriverID, RS.constructorId
), CTE2 AS (
    SELECT 
        Year,
        DriverID,
        ConstructorID,
        SUM(CASE WHEN TotalPoints = MaxPointsInYear THEN 1 ELSE 0 END) AS TotalWorldChampionships
    FROM 
        CTE
    GROUP BY 
        Year, DriverID, ConstructorID
    ORDER BY 
        Year
)
SELECT CTE2.*, CONCAT(D.forename,' ',D.surname) AS DriverName, C.name AS ConstructorName FROM CTE2 
INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/drivers` AS D on D.driverid = CTE2.driverid
INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructors` AS C on C.constructorid = CTE2.constructorId
WHERE TotalWorldChampionships = 1
"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/YearlyChampions')

24/05/09 18:36:02 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 610577 ms exceeds timeout 120000 ms
24/05/09 18:36:02 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/09 18:36:02 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$